# Workflow to produce interactive web map from text source

- Use an annotation tool to tag placenames in a text. This example uses Recogito (https://recogito.pelagios.org/) and 'The Letters of Robert Burns', obtained as a plain text file from project Gutenberg (https://www.gutenberg.org/ebooks/9863). 

- Use a gazetteer to geocode the tagged placenames. Recogito has built-in support for the Geonames gazetter (https://www.geonames.org/) and this has been used in this example.

- Export or obtain the tagged placenames as a GeoJSON (https://geojson.org/) file, and use the file in this notebook. In this example the file used is called 'rburns1.json'. To use your own file upload it using the file manager in the lefthand menu bar and change the file name in cell B.1 below, or paste the data directly into cell B.2.

- This notebook uses ipyleaflet to create a web map using the data. Use the reference at https://ipyleaflet.readthedocs.io/en/latest/ to modify the example code in steps C.1 to C.4 to change the behaviour and appearance of the map. Step C.4 uses historical mapping data from the National Library of Scotland (https://maps.nls.uk/guides/georeferencing/).

**A. Import required modules**

In [ ]:
import json
from ipyleaflet import Map, GeoJSON, Marker, basemaps, basemap_to_tiles, ScaleControl, LayersControl, WidgetControl, LayerGroup, TileLayer
from ipywidgets import HTML, widgets

**B. Read placenames data**

B.1 Read data from uploaded file

In [ ]:
with open('rburns1.json', encoding='utf-8') as f:
    data = json.load(f)

In [ ]:
features = data['features']

B.2 Read data directly from GeoJSON text pasted into cell (replace everything in the example after `data =`)

In [ ]:
data = {"type":"FeatureCollection","features":[{"type":"Feature","geometry":{"type":"Point","coordinates":[-3.19648,55.95206]},"properties":{"titles":"Edinburgh","annotations":48},"uris":["http://sws.geonames.org/2650225"],"titles":["Edinburgh"],"names":["Edinburgh","Edinburgh","Edimburgo","Édimbourg","Edimburgo","エディンバラ","Edimburgo","Эдинбург","Εδιμβούργο","এডিনবরা","एडिनबरा","Edinburgh","إدنبرة","Aebura","爱丁堡"],"source_gazetteers":["http://www.geonames.org"],"quotes":["Edinburgh","Edinburgh","Edinburgh","Edinburgh","Edinburgh","Edinburgh","Edinburgh","Edinburgh","Edinburgh","Edinburgh","Edinburgh","Edinburgh","Edinburgh","Edinburgh","Edinburgh","Edinburgh","Edinburgh","Edinburgh","Edinburgh","Edinburgh","Edinburgh","Edinburgh","Edinburgh","Edinburgh","Edinburgh","Edinburgh","Edinburgh","Edinburgh","Edinburgh","Edinburgh","Edinburgh","Edinburgh","Edinburgh","Edinburgh","Edinburgh","Edinburgh","Edinburgh","Edinburgh","Edinburgh","Edinburgh","Edinburgh","Edinburgh","Edinburgh","Edinburgh","Edinburgh","Edinburgh","Edinburgh","Edinburgh"]},{"type":"Feature","geometry":{"type":"Point","coordinates":[-3.61139,55.06959]},"properties":{"titles":"Dumfries","annotations":12},"uris":["http://sws.geonames.org/2650798"],"titles":["Dumfries"],"names":["Дамфрис","ダンフリーズ","鄧弗里斯"],"source_gazetteers":["http://www.geonames.org"],"quotes":["Dumfries","Dumfries","Dumfries","Dumfries","Dumfries","Dumfries","Dumfries","Dumfries","Dumfries","Dumfries","Dumfries","Dumfries"]},{"type":"Feature","geometry":{"type":"Point","coordinates":[-4.25763,55.86515]},"properties":{"titles":"Glasgow","annotations":6},"uris":["http://sws.geonames.org/2648579"],"titles":["Glasgow"],"names":["Glasgow","Glasgow","Glasgow","Γλασκώβη","Glasgow","Glasgow","グラスゴー","Glasgow","Глазго","格拉斯哥","ग्लासगो","غلاسكو","গ্লাসগো","ਗਲਾਸਗੋ"],"source_gazetteers":["http://www.geonames.org"],"quotes":["Glasgow","Glasgow","Glasgow","Glasgow","Glasgow","Glasgow"]},{"type":"Feature","geometry":{"type":"Point","coordinates":[-4.26644,55.45445]},"properties":{"titles":"Cumnock","annotations":1},"uris":["http://sws.geonames.org/2651706"],"titles":["Cumnock"],"names":["Cumnock"],"source_gazetteers":["http://www.geonames.org"],"quotes":["CUMNOCK"]}]}

In [ ]:
features = data['features']

**C. Generate and configure map**

C.1 Basic map (uses OpenStreetMap).

In [ ]:
center = (55, -3)

map1 = Map(
    center=(center),
    zoom=6
    )

map1.add_control(ScaleControl(position='bottomleft'))

map1

C.2 Map with marker icons using the supplied placenames data (click on each marker to see the placename).

In [ ]:
center = (55, -3)

map2 = Map(
    center=(center),
    zoom=6
    )

layer_group = LayerGroup(name='places')
    
for i in range(len(features)):
    location=(features[i]['geometry']['coordinates'][1],features[i]['geometry']['coordinates'][0])
    placenames = features[i]['properties']['titles']
    html = """
    <p>""" + placenames + """</p>
    """
    marker = Marker(location=location, draggable=False)
    marker.popup = HTML(html)
    layer_group.add_layer(marker)
    
map2.add_layer(layer_group)
map2.add_control(ScaleControl(position='bottomleft'))

map2

C.3 Map with basemap/marker display selection control. This example adds OpenStreetMap topo and ESRI World maps, see https://ipyleaflet.readthedocs.io/en/latest/map_and_basemaps/basemaps.html for more map options).

In [ ]:
center = (55, -3)

map3 = Map(
    center=(center),
    zoom=6
    )

layer_group = LayerGroup(name='places')

for i in range(len(features)):
    location=(features[i]['geometry']['coordinates'][1],features[i]['geometry']['coordinates'][0])
    placenames = features[i]['properties']['titles']
    html = """
    <p>""" + placenames + """</p>
    """
    marker = Marker(location=location, draggable=False)
    marker.popup = HTML(html)
    layer_group.add_layer(marker)
    
map3.add_layer(layer_group)

topo = basemap_to_tiles(basemaps.OpenTopoMap)
topo.base = True
topo.name = 'Open Topo Layer'

esri = basemap_to_tiles(basemaps.Esri.WorldImagery)
esri.base = True
esri.name = 'ESRI World layer'

map3.add_layer(topo)
map3.add_layer(esri)
map3.add_control(LayersControl(position='topright'))
map3.add_control(ScaleControl(position='bottomleft'))

map3

C.4 Map with a NLS historical mapping layer. This example uses Ordnance Survey Great Britain One Inch 1885-1900 mapping. For other mapping layers available from the NLS see https://maps.nls.uk/guides/georeferencing/layers-list/. To use one of these layers find the URL for the layer using the steps at https://maps.nls.uk/guides/georeferencing/layers-urls/ and insert it into the line `url1 = ""` below. A slider control to alter the opacity of the layer is also added.

In [ ]:
center = (55, -3)

map4 = Map(
    center=(center),
    zoom=6
    )

layer_group = LayerGroup(name='places')

for i in range(len(features)):
    location=(features[i]['geometry']['coordinates'][1],features[i]['geometry']['coordinates'][0])
    placenames = features[i]['properties']['titles']
    html = """
    <p>""" + placenames + """</p>
    """
    marker = Marker(location=location, draggable=False)
    marker.popup = HTML(html)
    layer_group.add_layer(marker)

map4.add_layer(layer_group)

url1 = "https://mapseries-tilesets.s3.amazonaws.com/1inch_2nd_ed/{z}/{x}/{y}.png"
nls1 = TileLayer(url=url1, opacity=1)

map4.add_layer(nls1)
map4.add_control(ScaleControl(position='bottomleft'))

slider = widgets.FloatSlider(min=0, max=1, value=1, readout=False, layout=widgets.Layout(width='15em'))
widgets.jslink((slider, 'value'),(nls1, 'opacity'))
map4.add_control(WidgetControl(widget=slider, position='topright'))

map4

**D. Save map as HTML file**

In [ ]:
map4.save('my_map.html', title='My Map')